### COCO Dataset :  train , validation and test data dumping

In [1]:
import json
import pickle
import glob
import pickle
root='D:/COCO/'
#f=open(,'r')
groups=['train','val','test']
files=[root+'annot_2017/instances_train2017.json',
       root+'annot_2017/instances_val2017.json',
       root+'annot_2017/image_info_test-dev2017.json']

img_files=[root+'train2017/train2017/*',
           root+'val2017/val2017/*',
           root+'test2017/test2017/*']

annot_dicts=[]
for file in files:
    f=open(file,'r')
    annot_dict=json.load(f)
    annot_dicts.append(annot_dict)

In [2]:
LABELS={}
for cat in annot_dicts[0]['categories']:
    LABELS[cat['id']]=cat['name']

In [3]:
f=open(root+'labels_coco','wb')
pickle.dump(LABELS,f)
f.close()

In [4]:
def save_annot(annot_dict,img_file,group):
    
    images=annot_dict['images']
    images_dict={}
    if group !='test': 
        annotations=annot_dict['annotations']
        for image in images:
            images_dict[image['id']]=[image['file_name'],image['width'],image['height']]
        annots_dict={}
        for annot in annotations:
            annots_dict[annot['image_id']]=[]
        for annot in annotations:
            annots_dict[annot['image_id']].append(annot)
        train_images={}
        for _id in annots_dict.keys():
            train_img={}
            image=images_dict[_id]
            annots=annots_dict[_id]
            train_img['id']=_id
            train_img['filename']=image[0]
            train_img['width']=image[1]
            train_img['height']=image[2]
            objs=[]
            wf=1/image[1]
            hf=1/image[2]
            for annot in annots:
                if annot['iscrowd']==0:
                   obj={}
                   bbox=annot['bbox']
                   obj['name']=LABELS[annot['category_id']]
                   obj['xmin']=bbox[0]*wf
                   obj['ymin']=bbox[1]*hf
                   obj['xmax']=(bbox[0]+bbox[2])*wf
                   obj['ymax']=(bbox[1]+bbox[3])*hf
                   objs.append(obj)
            train_img['object']=objs
            train_images[image[0]]=train_img
        fnames=[]  
        _data=[] 
        for i,file in enumerate(glob.glob(img_file)):
                file_name=file.split('\\')[-1]
                try:data=train_images[file_name]
                except:continue
                _data.append(data)
    if group=='test':
        _data=[]
        for i,data in enumerate(images):
            data['filename']=data['file_name']
            _data.append(data)
    print('saving files','len ',len(_data),'fname ',root+group+'_coco_dfs')
    f=open(root+group+'_coco_dfs','wb')
    pickle.dump(_data,f)
    f.close()

In [5]:
for annot_dict,img_file,group in zip(annot_dicts,img_files,groups):
                   save_annot(annot_dict,img_file,group)

saving files len  117266 fname  D:/COCO/train_coco_dfs
saving files len  4952 fname  D:/COCO/val_coco_dfs
saving files len  20288 fname  D:/COCO/test_coco_dfs


### Extra functions

In [6]:
import csv

def read_data(dataset):
  f=open(dataset+'/label.csv')
  file=csv.reader(f,delimiter=',')
  data=[]
  i=0
  sc=416.0
  for line in file:
      dt=line
      H=dt[0]
      W=dt[1]
      xmin=(float(dt[2]))/sc
      ymin=(float(dt[3]))/sc
      xmax=(float(dt[4]))/sc
      ymax=(float(dt[5]))/sc

      output={
          'filename':dataset+'/images/'+str(i)+'.jpg',
          'height':H,
          'width':W,
          'object':[{'name':'None',
          'xmin':xmin*IMAGE_W,
          'ymin':ymin*IMAGE_H,
          'xmax':xmax*IMAGE_W,
          'ymax':ymax*IMAGE_H}]
          }

      data.append(output)
      i=i+1
  return data
